In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from hmmlearn import hmm 
from scipy.signal import butter, filtfilt

def butter_bandpass_filter(data, lowcut, highcut, fs, order = 5):
    #fs - sampling rate
    fa = 0.5 * fs
    low = lowcut/fa
    high = highcut/fa
    b,a = butter(order,[low, high], btype='band')
    y = filtfilt(b,a,data)
    return y

def draw_pic(data, long = 10, high = 10):
    x= data.shape[0]
    x_linspace = np.arange(0,x)
    #data = data - np.mean(data)
    plt.figure(figsize=(long,high))
    plt.plot(x_linspace,data)
    return

In [ ]:
data = sio.loadmat('E:/study/Doctor/second_year/EEG_data/UCLA_data/times_CSC4.mat')
spikes = data['spikes']
exp_data2 = spikes.T[1]
filtered_signal = butter_bandpass_filter(exp_data2,100,6000,48000)
draw_pic(filtered_signal,90,20)

In [ ]:
data = filtered_signal
K = 30
T = data.shape[0]
cov = np.cov(data)
std = np.std(data)
pi = np.random.dirichlet(np.ones(30),size = 1).reshape(-1,1)
P = 10**(-3) 

mu_sr = 24 * 10 ** 3
lins = np.linspace(0,3 * np.pi , num = mu_sr)
sinmu = np.sin(lins)
mu_linspace = np.zeros((K,1))

for i in range(5,14):
    mu_linspace[i,0] = sinmu[int((i-5) * mu_sr / 10)]

h_mu = np.random.uniform(0,5*std)
mu = mu_linspace * np.matrix(h_mu)

In [ ]:
tran_pro = np.append(np.zeros([1,K]),np.append(np.eye(K-1,dtype=float),np.zeros([K-1,1]),axis=1),axis=0)
tran_pro[0,0],tran_pro[1,0],tran_pro[0,-1] = 1-P, P, 1

In [ ]:
model = hmm.GaussianHMM(K,covariance_type="spherical")

In [ ]:
model.transmat_prior = tran_pro
model.means_prior = mu
model.covars_ = np.ones((K)) * cov

In [ ]:
logprob,state = model.decode(data,algorithm = 'viterbi')